### Development and example of patch-to-point extraction workflow

In [1]:
import openeo
import pandas as pd 
import pystac
import pystac_client

from shapely.geometry import shape, MultiPolygon

from worldcereal.extract.patch_to_point_worldcereal import create_job_patch_to_point_worldcereal, get_sample_points_from_rdm
from worldcereal.rdm_api import RdmInteraction

#### Create job dataframe

As discussed we will orchestrate the jobs by splittin per `ref_id` and `EPSG`. The below will be replaced by a `create_job_dataframe_patch_to_point_worldcereal` function (or a fancier name). For now we just create a dummy `pandas.DataFrame` containing all necessary columns (prone to change).

In [2]:
job_df = pd.read_parquet('job_df.parquet')
job_df

,backend,start_date,end_date,epsg,ref_id
0,Terrascope,2021-01-01,2022-01-01,32633,2021_AUT_LPIS_POLY_110


Now we find all samples in RDM that match the following:

- ref_id = ref_id
- Lie within patch extraction patches (polygons that are partly within are cut off)
- Temporal extent matches given temporal extent

We can either include the `get_sample_points_from_rdm` in the `create_job` function, but this creates extra overhead before submitting the job. Otherwise, we can make it part of the `create_job_dataframe` workflow (fetch gdf and upload to artifactory as geoparquet, and include artifactory URL in job database).

This returns all geometries in the RDM, cut-off so that they are fully within the patches, and then taken the centroid. 



#### Create job patch to point

Here we create the openEO process graph to be sent to the backend

In [3]:
row = job_df.iloc[0]
connection = openeo.connect('openeo.vito.be').authenticate_oidc()

Authenticated using refresh token.


In [4]:
job = create_job_patch_to_point_worldcereal(
    row=row,
    connection=connection,
    provider=None,
    connection_provider=None,
    executor_memory='2g',
    python_memory='2g',
    max_executors=20

)

2025-03-11 13:42:07.883 | INFO     | worldcereal.rdm_api.rdm_interaction:get_samples:500 - Querying 1 collections...


ValueError: Artifactory credentials not found. Please set ARTIFACTORY_USERNAME and ARTIFACTORY_PASSWORD.